In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!sudo apt install neofetch -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  caca-utils chafa fonts-droid-fallback fonts-noto-mono fonts-urw-base35
  ghostscript gsfonts imagemagick imagemagick-6-common imagemagick-6.q16 jp2a
  libchafa0 libdjvulibre-text libdjvulibre21 libfftw3-double3 libgs9
  libgs9-common libid3tag0 libidn12 libijs-0.35 libimlib2 libjbig2dec0
  libjxr-tools libjxr0 liblqr-1-0 libmagickcore-6.q16-6
  libmagickcore-6.q16-6-extra libmagickwand-6.q16-6 libnetpbm10 libpci3
  libsixel-bin libsixel1 libwmflite-0.2-7 netpbm pci.ids pciutils poppler-data
  toilet toilet-fonts w3m w3m-img
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf fonts-texgyre
  ghostscript-x imagemagick-doc autotrace cups-bsd | lpr | lprng enscript gimp
  gnuplot grads hp2xx html2ps libwmf-bin mplayer povray radiance sane-utils
  texlive-base-bin transfig ufraw-batch libfftw3-bin libfftw3-dev inkscape
 

In [3]:
!neofetch

            .-/+oossssoo+/-.
        `:+ssssssssssssssssss+:`
      -+ssssssssssssssssssyyssss+-
    .ossssssssssssssssssdMMMNysssso.
   /ssssssssssshdmmNNmmyNMMMMhssssss/
  +ssssssssshmydMMMMMMMNddddyssssssss+
 /sssssssshNMMMyhhyyyyhmNMMMNhssssssss/
.ssssssssdMMMNhsssssssssshNMMMdssssssss.
+sssshhhyNMMNyssssssssssssyNMMMysssssss+
ossyNMMMNyMMhsssssssssssssshmmmhssssssso
ossyNMMMNyMMhsssssssssssssshmmmhssssssso
+sssshhhyNMMNyssssssssssssyNMMMysssssss+
.ssssssssdMMMNhsssssssssshNMMMdssssssss.
 /sssssssshNMMMyhhyyyyhdNMMMNhssssssss/
  +sssssssssdmydMMMMMMMMddddyssssssss+
   /ssssssssssshdmNNNNmyNMMMMhssssss/
    .ossssssssssssssssssdMMMNysssso.
      -+sssssssssssssssssyyyssss+-
        `:+ssssssssssssssssss+:`
            .-/+oossssoo+/-.
root@0651b8b13195 
----------------- 
OS: Ubuntu 22.04.4 LTS x86_64 
Host: Google Compute Engine 
Kernel: 6.1.85+ 
Uptime: 3 mins 
Packages: 1366 (dpkg) 
Shell: bash 5.1.16 
Terminal: jupyter-noteboo 
CPU: Intel Xeon (2) @ 2.199GHz 
GPU: NVIDIA Tesla T

## Thêm 600 ảnh bạo lực

In [4]:
import os
import shutil
import random
import numpy as np
from google.colab import drive
import yaml
from tqdm import tqdm

In [5]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
images_folder = '/content/drive/MyDrive/ImageGuard/data_yolo/train/images'
labels_folder = '/content/drive/MyDrive/ImageGuard/data_yolo/train/labels'
dataset_folder = '/content/drive/MyDrive/ImageGuard/dataset'

Tạo cấu trúc thư mục    

In [7]:
def create_directory_structure(base_path):
    directories = [
        os.path.join(base_path, 'train', 'images'),
        os.path.join(base_path, 'train', 'labels'),
        os.path.join(base_path, 'val', 'images'),
        os.path.join(base_path, 'val', 'labels'),
        os.path.join(base_path, 'test', 'images'),
        os.path.join(base_path, 'test', 'labels')
    ]

    for directory in directories:
        os.makedirs(directory, exist_ok=True)
        print(f"Đã tạo thư mục: {directory}")


Lấy danh sách các file ảnh và nhãn tương ứng

In [8]:
def get_matching_files(images_dir, labels_dir):
    # Lấy tất cả các file ảnh
    image_files = [f for f in os.listdir(images_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]

    # Kiểm tra xem mỗi ảnh có file nhãn tương ứng không
    valid_pairs = []
    for img_file in image_files:
        # Lấy tên file không có phần mở rộng
        base_name = os.path.splitext(img_file)[0]
        label_file = f"{base_name}.txt"

        if os.path.exists(os.path.join(labels_dir, label_file)):
            valid_pairs.append((img_file, label_file))

    return valid_pairs

Chia tập dữ liệu theo tỷ lệ

In [9]:
def split_dataset(file_pairs, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15, total_images=600):
    # Đảm bảo tổng cộng số ảnh là 600
    if len(file_pairs) > total_images:
        print(f"Giới hạn số lượng cặp ảnh-nhãn xuống {total_images} từ {len(file_pairs)}")
        random.shuffle(file_pairs)
        file_pairs = file_pairs[:total_images]
    elif len(file_pairs) < total_images:
        print(f"Cảnh báo: Chỉ có {len(file_pairs)} cặp ảnh-nhãn hợp lệ, ít hơn yêu cầu {total_images}")

    # Shuffle dữ liệu
    random.shuffle(file_pairs)

    # Tính số lượng cho mỗi tập
    n_train = int(len(file_pairs) * train_ratio)
    n_val = int(len(file_pairs) * val_ratio)

    # Chia tập dữ liệu
    train_pairs = file_pairs[:n_train]
    val_pairs = file_pairs[n_train:n_train + n_val]
    test_pairs = file_pairs[n_train + n_val:]

    print(f"Số lượng cặp ảnh-nhãn cho train: {len(train_pairs)}")
    print(f"Số lượng cặp ảnh-nhãn cho validation: {len(val_pairs)}")
    print(f"Số lượng cặp ảnh-nhãn cho test: {len(test_pairs)}")

    return train_pairs, val_pairs, test_pairs

Sao chép file vào cấu trúc thư mục mới

In [10]:
def copy_files(file_pairs, src_images_dir, src_labels_dir, dst_base_dir, subset):
    dst_images_dir = os.path.join(dst_base_dir, subset, 'images')
    dst_labels_dir = os.path.join(dst_base_dir, subset, 'labels')

    for img_file, label_file in tqdm(file_pairs, desc=f"Đang sao chép file vào {subset}"):
        # Sao chép ảnh
        src_img_path = os.path.join(src_images_dir, img_file)
        dst_img_path = os.path.join(dst_images_dir, img_file)
        shutil.copy2(src_img_path, dst_img_path)

        # Sao chép nhãn
        src_label_path = os.path.join(src_labels_dir, label_file)
        dst_label_path = os.path.join(dst_labels_dir, label_file)
        shutil.copy2(src_label_path, dst_label_path)

Thực thi các hàm

In [12]:
create_directory_structure(dataset_folder)

print("Đang tìm các cặp file ảnh và nhãn tương ứng...")
file_pairs = get_matching_files(images_folder, labels_folder)
print(f"Tìm thấy {len(file_pairs)} cặp file ảnh-nhãn hợp lệ")

train_pairs, val_pairs, test_pairs = split_dataset(file_pairs, total_images=600)

print("Đang sao chép dữ liệu vào thư mục train...")
copy_files(train_pairs, images_folder, labels_folder, dataset_folder, 'train')

print("Đang sao chép dữ liệu vào thư mục validation...")
copy_files(val_pairs, images_folder, labels_folder, dataset_folder, 'val')

print("Đang sao chép dữ liệu vào thư mục test...")
copy_files(test_pairs, images_folder, labels_folder, dataset_folder, 'test')

print("Hoàn thành việc chuẩn bị dataset!")

Đã tạo thư mục: /content/drive/MyDrive/ImageGuard/dataset/train/images
Đã tạo thư mục: /content/drive/MyDrive/ImageGuard/dataset/train/labels
Đã tạo thư mục: /content/drive/MyDrive/ImageGuard/dataset/val/images
Đã tạo thư mục: /content/drive/MyDrive/ImageGuard/dataset/val/labels
Đã tạo thư mục: /content/drive/MyDrive/ImageGuard/dataset/test/images
Đã tạo thư mục: /content/drive/MyDrive/ImageGuard/dataset/test/labels
Đang tìm các cặp file ảnh và nhãn tương ứng...
Tìm thấy 6005 cặp file ảnh-nhãn hợp lệ
Giới hạn số lượng cặp ảnh-nhãn xuống 600 từ 6005
Số lượng cặp ảnh-nhãn cho train: 420
Số lượng cặp ảnh-nhãn cho validation: 90
Số lượng cặp ảnh-nhãn cho test: 90
Đang sao chép dữ liệu vào thư mục train...


Đang sao chép file vào train: 100%|██████████| 420/420 [04:44<00:00,  1.48it/s]


Đang sao chép dữ liệu vào thư mục validation...


Đang sao chép file vào val: 100%|██████████| 90/90 [00:34<00:00,  2.59it/s]


Đang sao chép dữ liệu vào thư mục test...


Đang sao chép file vào test: 100%|██████████| 90/90 [00:35<00:00,  2.56it/s]

Không xác định được tên các lớp. Sử dụng tên mặc định 'class0', 'class1', ...


IsADirectoryError: [Errno 21] Is a directory: '/content/drive/MyDrive/ImageGuard/dataset/data.yaml'

## Thêm 600 ảnh bình thường

In [15]:
# Thêm ảnh và tạo nhãn toàn ảnh với class 0
import os
import shutil
import random
from tqdm import tqdm
import numpy as np
from PIL import Image
import yaml
from google.colab import drive

# Mount Google Drive (nếu chưa mount)
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# Định nghĩa các đường dẫn
images_source_folder = '/content/drive/MyDrive/ImageGuard/images'
dataset_folder = '/content/drive/MyDrive/ImageGuard/dataset'

# Kiểm tra và tạo cấu trúc thư mục nếu chưa tồn tại
def create_directory_structure(base_path):
    directories = [
        os.path.join(base_path, 'train', 'images'),
        os.path.join(base_path, 'train', 'labels'),
        os.path.join(base_path, 'val', 'images'),
        os.path.join(base_path, 'val', 'labels'),
        os.path.join(base_path, 'test', 'images'),
        os.path.join(base_path, 'test', 'labels')
    ]

    for directory in directories:
        os.makedirs(directory, exist_ok=True)
        print(f"Đã tạo hoặc xác nhận thư mục: {directory}")

# Hàm tạo nhãn với toàn bộ ảnh được coi là một đối tượng với class 0
def create_full_image_label(image_path, label_path):
    try:
        # Đọc kích thước ảnh
        with Image.open(image_path) as img:
            width, height = img.size

        # Tạo nhãn YOLO format (class x_center y_center width height)
        # Với tất cả là class 0 và kích thước bằng toàn bộ ảnh
        # YOLO format là tương đối so với kích thước ảnh (từ 0 đến 1)
        x_center, y_center = 0.5, 0.5  # Trung tâm ảnh
        w, h = 1.0, 1.0  # Chiều rộng và chiều cao tương đối là 1.0 (toàn bộ ảnh)

        # Ghi nhãn vào file
        with open(label_path, 'w') as f:
            f.write(f"0 {x_center} {y_center} {w} {h}")

        return True
    except Exception as e:
        print(f"Lỗi khi tạo nhãn cho ảnh {image_path}: {e}")
        return False

# Lấy danh sách các file ảnh
def get_image_files(images_dir, max_count=600):
    # Lấy tất cả các file ảnh
    image_files = [f for f in os.listdir(images_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]

    # Giới hạn số lượng ảnh
    if len(image_files) > max_count:
        random.shuffle(image_files)
        image_files = image_files[:max_count]
        print(f"Đã chọn ngẫu nhiên {max_count} ảnh từ tổng số {len(image_files)} ảnh.")
    else:
        print(f"Sử dụng tất cả {len(image_files)} ảnh có sẵn.")

    return image_files

# Chia tập dữ liệu theo tỷ lệ
def split_dataset(file_list, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):
    # Shuffle dữ liệu
    random.shuffle(file_list)

    # Tính số lượng cho mỗi tập
    n_train = int(len(file_list) * train_ratio)
    n_val = int(len(file_list) * val_ratio)

    # Chia tập dữ liệu
    train_files = file_list[:n_train]
    val_files = file_list[n_train:n_train + n_val]
    test_files = file_list[n_train + n_val:]

    print(f"Số lượng ảnh cho train: {len(train_files)}")
    print(f"Số lượng ảnh cho validation: {len(val_files)}")
    print(f"Số lượng ảnh cho test: {len(test_files)}")

    return train_files, val_files, test_files

# Sao chép ảnh và tạo nhãn
def process_and_copy_files(file_list, src_dir, dst_base_dir, subset):
    dst_images_dir = os.path.join(dst_base_dir, subset, 'images')
    dst_labels_dir = os.path.join(dst_base_dir, subset, 'labels')

    processed_count = 0

    for img_file in tqdm(file_list, desc=f"Đang xử lý và sao chép file vào {subset}"):
        # Đường dẫn nguồn và đích cho ảnh
        src_img_path = os.path.join(src_dir, img_file)
        dst_img_path = os.path.join(dst_images_dir, img_file)

        # Tên file nhãn tương ứng
        base_name = os.path.splitext(img_file)[0]
        label_file = f"{base_name}.txt"
        dst_label_path = os.path.join(dst_labels_dir, label_file)

        # Sao chép ảnh
        try:
            shutil.copy2(src_img_path, dst_img_path)

            # Tạo nhãn
            if create_full_image_label(src_img_path, dst_label_path):
                processed_count += 1
        except Exception as e:
            print(f"Lỗi khi xử lý file {img_file}: {e}")

    return processed_count



# Thực hiện quy trình
def main():
    print("Bắt đầu tiến trình tạo dataset...")

    # Tạo cấu trúc thư mục
    create_directory_structure(dataset_folder)

    # Lấy danh sách file ảnh
    print("Đang tìm các file ảnh...")
    image_files = get_image_files(images_source_folder, max_count=600)

    # Chia tập dữ liệu
    train_files, val_files, test_files = split_dataset(image_files)

    # Xử lý và sao chép file
    print("Đang xử lý tập train...")
    train_processed = process_and_copy_files(train_files, images_source_folder, dataset_folder, 'train')

    print("Đang xử lý tập validation...")
    val_processed = process_and_copy_files(val_files, images_source_folder, dataset_folder, 'val')

    print("Đang xử lý tập test...")
    test_processed = process_and_copy_files(test_files, images_source_folder, dataset_folder, 'test')



    # Thống kê
    total_processed = train_processed + val_processed + test_processed
    print(f"\nTổng kết:")
    print(f"- Đã xử lý thành công: {total_processed}/{len(image_files)} ảnh")
    print(f"- Train: {train_processed} ảnh")
    print(f"- Validation: {val_processed} ảnh")
    print(f"- Test: {test_processed} ảnh")
    print("\nHoàn thành việc xử lý và tạo dataset!")

# Thực thi chương trình
if __name__ == "__main__":
    main()

Bắt đầu tiến trình tạo dataset...
Đã tạo hoặc xác nhận thư mục: /content/drive/MyDrive/ImageGuard/dataset/train/images
Đã tạo hoặc xác nhận thư mục: /content/drive/MyDrive/ImageGuard/dataset/train/labels
Đã tạo hoặc xác nhận thư mục: /content/drive/MyDrive/ImageGuard/dataset/val/images
Đã tạo hoặc xác nhận thư mục: /content/drive/MyDrive/ImageGuard/dataset/val/labels
Đã tạo hoặc xác nhận thư mục: /content/drive/MyDrive/ImageGuard/dataset/test/images
Đã tạo hoặc xác nhận thư mục: /content/drive/MyDrive/ImageGuard/dataset/test/labels
Đang tìm các file ảnh...
Đã chọn ngẫu nhiên 600 ảnh từ tổng số 600 ảnh.
Số lượng ảnh cho train: 420
Số lượng ảnh cho validation: 90
Số lượng ảnh cho test: 90
Đang xử lý tập train...


Đang xử lý và sao chép file vào train: 100%|██████████| 420/420 [00:17<00:00, 23.97it/s]


Đang xử lý tập validation...


Đang xử lý và sao chép file vào val: 100%|██████████| 90/90 [00:01<00:00, 51.71it/s]


Đang xử lý tập test...


Đang xử lý và sao chép file vào test: 100%|██████████| 90/90 [00:01<00:00, 48.48it/s]


Tổng kết:
- Đã xử lý thành công: 600/600 ảnh
- Train: 420 ảnh
- Validation: 90 ảnh
- Test: 90 ảnh

Hoàn thành việc xử lý và tạo dataset!


## Thêm 600 ảnh người lớn

In [16]:
# Thêm ảnh và tạo nhãn toàn ảnh với class 2
import os
import shutil
import random
from tqdm import tqdm
import numpy as np
from PIL import Image
import yaml
from google.colab import drive

# Mount Google Drive (nếu chưa mount)
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# Định nghĩa các đường dẫn
images_source_folder = '/content/drive/MyDrive/ImageGuard/sex_photos'
dataset_folder = '/content/drive/MyDrive/ImageGuard/dataset'

# Kiểm tra và tạo cấu trúc thư mục nếu chưa tồn tại
def create_directory_structure(base_path):
    directories = [
        os.path.join(base_path, 'train', 'images'),
        os.path.join(base_path, 'train', 'labels'),
        os.path.join(base_path, 'val', 'images'),
        os.path.join(base_path, 'val', 'labels'),
        os.path.join(base_path, 'test', 'images'),
        os.path.join(base_path, 'test', 'labels')
    ]

    for directory in directories:
        os.makedirs(directory, exist_ok=True)
        print(f"Đã tạo hoặc xác nhận thư mục: {directory}")

# Hàm tạo nhãn với toàn bộ ảnh được coi là một đối tượng với class 0
def create_full_image_label(image_path, label_path):
    try:
        # Đọc kích thước ảnh
        with Image.open(image_path) as img:
            width, height = img.size

        # Tạo nhãn YOLO format (class x_center y_center width height)
        # Với tất cả là class 0 và kích thước bằng toàn bộ ảnh
        # YOLO format là tương đối so với kích thước ảnh (từ 0 đến 1)
        x_center, y_center = 0.5, 0.5  # Trung tâm ảnh
        w, h = 1.0, 1.0  # Chiều rộng và chiều cao tương đối là 1.0 (toàn bộ ảnh)

        # Ghi nhãn vào file
        with open(label_path, 'w') as f:
            f.write(f"2 {x_center} {y_center} {w} {h}")

        return True
    except Exception as e:
        print(f"Lỗi khi tạo nhãn cho ảnh {image_path}: {e}")
        return False

# Lấy danh sách các file ảnh
def get_image_files(images_dir, max_count=600):
    # Lấy tất cả các file ảnh
    image_files = [f for f in os.listdir(images_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]

    # Giới hạn số lượng ảnh
    if len(image_files) > max_count:
        random.shuffle(image_files)
        image_files = image_files[:max_count]
        print(f"Đã chọn ngẫu nhiên {max_count} ảnh từ tổng số {len(image_files)} ảnh.")
    else:
        print(f"Sử dụng tất cả {len(image_files)} ảnh có sẵn.")

    return image_files

# Chia tập dữ liệu theo tỷ lệ
def split_dataset(file_list, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):
    # Shuffle dữ liệu
    random.shuffle(file_list)

    # Tính số lượng cho mỗi tập
    n_train = int(len(file_list) * train_ratio)
    n_val = int(len(file_list) * val_ratio)

    # Chia tập dữ liệu
    train_files = file_list[:n_train]
    val_files = file_list[n_train:n_train + n_val]
    test_files = file_list[n_train + n_val:]

    print(f"Số lượng ảnh cho train: {len(train_files)}")
    print(f"Số lượng ảnh cho validation: {len(val_files)}")
    print(f"Số lượng ảnh cho test: {len(test_files)}")

    return train_files, val_files, test_files

# Sao chép ảnh và tạo nhãn
def process_and_copy_files(file_list, src_dir, dst_base_dir, subset):
    dst_images_dir = os.path.join(dst_base_dir, subset, 'images')
    dst_labels_dir = os.path.join(dst_base_dir, subset, 'labels')

    processed_count = 0

    for img_file in tqdm(file_list, desc=f"Đang xử lý và sao chép file vào {subset}"):
        # Đường dẫn nguồn và đích cho ảnh
        src_img_path = os.path.join(src_dir, img_file)
        dst_img_path = os.path.join(dst_images_dir, img_file)

        # Tên file nhãn tương ứng
        base_name = os.path.splitext(img_file)[0]
        label_file = f"{base_name}.txt"
        dst_label_path = os.path.join(dst_labels_dir, label_file)

        # Sao chép ảnh
        try:
            shutil.copy2(src_img_path, dst_img_path)

            # Tạo nhãn
            if create_full_image_label(src_img_path, dst_label_path):
                processed_count += 1
        except Exception as e:
            print(f"Lỗi khi xử lý file {img_file}: {e}")

    return processed_count



# Thực hiện quy trình
def main():
    print("Bắt đầu tiến trình tạo dataset...")

    # Tạo cấu trúc thư mục
    create_directory_structure(dataset_folder)

    # Lấy danh sách file ảnh
    print("Đang tìm các file ảnh...")
    image_files = get_image_files(images_source_folder, max_count=600)

    # Chia tập dữ liệu
    train_files, val_files, test_files = split_dataset(image_files)

    # Xử lý và sao chép file
    print("Đang xử lý tập train...")
    train_processed = process_and_copy_files(train_files, images_source_folder, dataset_folder, 'train')

    print("Đang xử lý tập validation...")
    val_processed = process_and_copy_files(val_files, images_source_folder, dataset_folder, 'val')

    print("Đang xử lý tập test...")
    test_processed = process_and_copy_files(test_files, images_source_folder, dataset_folder, 'test')



    # Thống kê
    total_processed = train_processed + val_processed + test_processed
    print(f"\nTổng kết:")
    print(f"- Đã xử lý thành công: {total_processed}/{len(image_files)} ảnh")
    print(f"- Train: {train_processed} ảnh")
    print(f"- Validation: {val_processed} ảnh")
    print(f"- Test: {test_processed} ảnh")
    print("\nHoàn thành việc xử lý và tạo dataset!")

# Thực thi chương trình
if __name__ == "__main__":
    main()

Bắt đầu tiến trình tạo dataset...
Đã tạo hoặc xác nhận thư mục: /content/drive/MyDrive/ImageGuard/dataset/train/images
Đã tạo hoặc xác nhận thư mục: /content/drive/MyDrive/ImageGuard/dataset/train/labels
Đã tạo hoặc xác nhận thư mục: /content/drive/MyDrive/ImageGuard/dataset/val/images
Đã tạo hoặc xác nhận thư mục: /content/drive/MyDrive/ImageGuard/dataset/val/labels
Đã tạo hoặc xác nhận thư mục: /content/drive/MyDrive/ImageGuard/dataset/test/images
Đã tạo hoặc xác nhận thư mục: /content/drive/MyDrive/ImageGuard/dataset/test/labels
Đang tìm các file ảnh...
Đã chọn ngẫu nhiên 600 ảnh từ tổng số 600 ảnh.
Số lượng ảnh cho train: 420
Số lượng ảnh cho validation: 90
Số lượng ảnh cho test: 90
Đang xử lý tập train...


Đang xử lý và sao chép file vào train: 100%|██████████| 420/420 [00:27<00:00, 15.40it/s]


Đang xử lý tập validation...


Đang xử lý và sao chép file vào val: 100%|██████████| 90/90 [00:03<00:00, 26.33it/s]


Đang xử lý tập test...


Đang xử lý và sao chép file vào test: 100%|██████████| 90/90 [00:02<00:00, 36.27it/s]


Tổng kết:
- Đã xử lý thành công: 600/600 ảnh
- Train: 420 ảnh
- Validation: 90 ảnh
- Test: 90 ảnh

Hoàn thành việc xử lý và tạo dataset!
